In [1]:
#0. Setup

import time
import sqlite3
import threading
from datetime import datetime
from matplotlib import pyplot
from matplotlib.dates import DateFormatter, AutoDateFormatter, AutoDateLocator
import matplotlib as plt
import json as json
import serial
import beebotte as beebotte
from beebotte import *
import paho.mqtt.client as mqtt

DBFILE = 'data.db'
BAUD = 57600
lines = {}
portname = "/dev/ttyUSB0"

port = serial.Serial(port=portname, baudrate=BAUD)  #connect to serial port on the MeLE mini PC

In [3]:
#1. Send signal to 5 nodes to start CO2 sensor calibration

def start_calibration_msg():
    data = {'type': 'start_calibration'}
    data_str = json.dumps(data).encode("utf-8")  #encode message
    print(data_str)
    port.write(data_str)  #send to serial port (to arduino explorer, to xbee s2c)

start_calibration_msg()

b'{"type": "start_calibration"}'


In [ ]:
#2. Accept information from nodes in JSON, and send to beebotte
from typing import Dict, Any

# Authenticate with Beebotte
bbt = BBT('UUtkSBvfRSUs4EhSsHL9JTwq', 'IMYOVMvxv5utLRdj5EXSJo9HJxmCbrAp')  #connect to beebotte with key and secret key

def read_message() -> Dict[str, Any]:  #function to read message from nodes
    while True:
        raw_message = port.read_until().decode().strip()
        try:
            return json.loads(raw_message)
        except json.JSONDecodeError:
            # Invalid message.
            print("Got invalid message, skipping.")

# Functions to send data
def send_temperature_humidity_data(channel: str, temperature: float, humidity: float) -> None:
    try:
        # Sending data to Beebotte
        bbt.write(channel, 'temperature', temperature)
        bbt.write(channel, 'humidity', humidity)
    except Exception as e:
        # Error handling
        print(f"An error occurred while sending data to channel {channel}: ", e)

def send_light_data(channel: str, light: float) -> None:
    try:
        # Sending data to Beebotte
        bbt.write(channel, 'light', light)
    except Exception as e:
        # Error handling
        print(f"An error occurred while sending data to channel {channel}: ", e)

def send_CO2_data(channel: str, CO2: float, VOC: float) -> None:
    try:
        # Sending data to Beebotte
        bbt.write(channel, 'CO2', CO2)
        bbt.write(channel, 'VOC', VOC)
    except Exception as e:
        # Error handling
        print(f"An error occurred while sending data to channel {channel}: ", e)
                                            
def main():
    while True:
        # Read the next message.
        message = read_message()

        channel = f"R{message['node']}"
        print(message)
        if message["type"] == "log":
            # Simply print out log messages to the console.
            print(f"LOG (Node {channel}): {message['message']}") 
        elif message["type"] == "temp":
            send_temperature_humidity_data(channel, message["temp"], message["humidity"])
        elif message["type"] == "air_qual":
            send_CO2_data(channel, message["co2"], message["voc"])
        elif message["type"] == "light":
            send_light_data(channel, message["intensity"])

main()


{'type': 'log', 'node': 1, 'message': 'Starting calibration.'}
LOG (Node R1): Starting calibration.
{'type': 'log', 'node': 2, 'message': 'Starting calibration.'}
LOG (Node R2): Starting calibration.
{'type': 'log', 'node': 3, 'message': 'Starting calibration.'}
LOG (Node R3): Starting calibration.
{'type': 'log', 'node': 4, 'message': 'Starting calibration.'}
LOG (Node R4): Starting calibration.
{'type': 'log', 'node': 5, 'message': 'Starting calibration.'}
LOG (Node R5): Starting calibration.
{'type': 'temp', 'node': 2, 'temp': 19.8, 'humidity': 12440.83}
{'type': 'air_qual', 'node': 2, 'co2': 516, 'voc': 0, 'calibrated': False}
{'type': 'light', 'node': 2, 'intensity': 0.005859}
{'type': 'temp', 'node': 5, 'temp': 19, 'humidity': 11707.13}
{'type': 'air_qual', 'node': 5, 'co2': 438, 'voc': 0, 'calibrated': False}
{'type': 'light', 'node': 5, 'intensity': 0.003906}
{'type': 'temp', 'node': 4, 'temp': 19, 'humidity': 12357.52}
{'type': 'air_qual', 'node': 4, 'co2': 436, 'voc': 0, 'cal

In [ ]:
#3. Send to beebotte in JSON

from beebotte import *

# Step 4: Sending Data to Multiple Channels
send_temperature_data('R1', 41)
send_temperature_data('R2', 44)
send_temperature_data('R3', 47)
send_temperature_data('R4', 51)
send_temperature_data('R5', 55)

    
